In [1]:
# 라이브러리 import
import os
import pandas as pd
import numpy as np
import math

# 셀레늄
from selenium.webdriver.common.alert import Alert
from selenium import webdriver  # 라이브러리(모듈) 가져오라
# pip install chromedriver_autoinstaller
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains as AC

from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# tqdm : for문 진행상황 체크
from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm

# 정규표현식(regular expression) : 문자(알파벳,한글), 숫자, 특수기호 정제 및 추출
import re
from time import sleep
import time

# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

# for문으로 크롤링 

In [95]:
# 검색어 입력
search = '삼성전자'

# 크롬 옵션
options = webdriver.ChromeOptions()

# 크롬 윈도우 사이즈 조절
options.add_argument("--window-size=1000,1200") # window-size -> 기본 : 1920,1080

# import chromedriver_autoinstaller

# chrome_path = chromedriver_autoinstaller.install()
# driver = webdriver.Chrome(chrome_path, options=options)
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

time.sleep(0.01)
driver.get("https://www.naver.com") # -> 네이버 크롬 창이 뜬다.

# 네이버 검색어 입력 후 검색
element = driver.find_element("name", "query") 
element.clear()  # 혹시 검색창에 존재하는 텍스트 제거

element.send_keys(search)  # 검색창에 검색어 전달
element.submit()  # 검색 클릭
time.sleep(0.01)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [/Users/krc/.wdm/drivers/chromedriver/mac64_m1/103.0.5060.134/chromedriver] found in cache


In [96]:
# 뉴스 클릭
driver.find_element("link text", "뉴스").click() 
time.sleep(0.01)

In [97]:
# '오래된 순' 클릭
driver.find_element("link text", "오래된순").click()  
time.sleep(0.01)

In [70]:
# 우선 수기로 옵션 누르고 조회 기간을 설정해줌

In [98]:
# 뉴스 데이터 수집
news_list = []
date_list = []
title_list = []
MAX_ITERATION = 350

for _ in range(120):
    time.sleep(1)

    # 기간 변경해 주기
    if len(date_list):
        splited_date = date_list[-1].split('.')
        year = int(splited_date[0]) - 1990 + 1
        month = int(splited_date[1])
        day = int(splited_date[2])

        # 기간을 클릭 해서 입력창 활성화  
        time.sleep(4)   
        driver.find_element(by=By.CSS_SELECTOR, value='#snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.option > a.txt.txt_option._calendar_select_trigger').click()
        time.sleep(2)

        ## 시작 일자 조정 ##
        
        # 년도 
        # 2016년 CSS_SELECTOR : snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.api_select_option.type_calendar._calendar_select_layer > div.select_wrap._root > div:nth-child(1) > div > div > div > ul > li:nth-child(27)
        driver.find_element(by=By.CSS_SELECTOR, value=f'#snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.api_select_option.type_calendar._calendar_select_layer > div.select_wrap._root > div:nth-child(1) > div > div > div > ul > li:nth-child({year})').click()
        time.sleep(1)
        
        # 월
        # 1월 CSS_SELECTOR : snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.api_select_option.type_calendar._calendar_select_layer > div.select_wrap._root > div:nth-child(2) > div > div > div > ul > li:nth-child(1)
        driver.find_element(by=By.CSS_SELECTOR, value=f'#snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.api_select_option.type_calendar._calendar_select_layer > div.select_wrap._root > div:nth-child(2) > div > div > div > ul > li:nth-child({month})').click()
        time.sleep(1)
        
        # 일
        # 1일 CSS_SELECTOR : #snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.api_select_option.type_calendar._calendar_select_layer > div.select_wrap._root > div:nth-child(3) > div > div > div > ul > li:nth-child(1)
        driver.find_element(by=By.CSS_SELECTOR, value=f'#snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.api_select_option.type_calendar._calendar_select_layer > div.select_wrap._root > div:nth-child(3) > div > div > div > ul > li:nth-child({day})').click()
        time.sleep(1)
        
        # "적용" 버튼 클릭
        driver.find_element(by=By.CSS_SELECTOR, value='#snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.api_select_option.type_calendar._calendar_select_layer > div.btn_area > button').click()
        time.sleep(2)
        
    for i in range(MAX_ITERATION):

        # 언론사
        newses = driver.find_elements('css selector', 'a.info.press')
        temp_list = []
        for news in newses:
            temp_list.append(news.text)
            temp_list = [x for x in temp_list ]
        news_list = news_list + temp_list 
        
        # 날짜
        dates = driver.find_elements('css selector', 'span.info')
        temp_list = []
        for date in dates:
            temp_list.append(date.text)
            temp_list = [x for x in temp_list ]
        date_list = date_list + temp_list 
        
        # 제목
        titles = driver.find_elements('css selector', 'a.news_tit')
        temp_list = []
        for title in titles:
            temp_list.append(title.text)
            temp_list = [x for x in temp_list ]
        title_list = title_list + temp_list 
        
        # 다음페이지 클릭
        driver.find_element('css selector', 'a.btn_next').click()
        time.sleep(0.001)

WebDriverException: Message: disconnected: Unable to receive message from renderer
  (Session info: chrome=103.0.5060.134)
Stacktrace:
0   chromedriver                        0x00000001003d1d60 chromedriver + 3792224
1   chromedriver                        0x0000000100368874 chromedriver + 3360884
2   chromedriver                        0x0000000100069184 chromedriver + 217476
3   chromedriver                        0x000000010005701c chromedriver + 143388
4   chromedriver                        0x00000001000567f8 chromedriver + 141304
5   chromedriver                        0x000000010006f768 chromedriver + 243560
6   chromedriver                        0x00000001000c1e40 chromedriver + 581184
7   chromedriver                        0x000000010008e3e4 chromedriver + 369636
8   chromedriver                        0x00000001003a7234 chromedriver + 3617332
9   chromedriver                        0x00000001003ab6c4 chromedriver + 3634884
10  chromedriver                        0x000000010

In [99]:
print(len(title_list))
print(news_list[-2:])
print(date_list[-2:])
print(title_list[-2:])

200400
['스포츠경향', '이데일리']
['2017.02.17.', '2017.02.17.']
['한정석 판사 “이재용 부회장 구속 사유 인정”…이유는?', "[맥모닝 뉴스] 한정석 판사 '이재용 구속영장' 발부, 한진해운 파산...직원 절..."]


In [ ]:
# df로 만들기
df = pd.DataFrame(list(zip(news_list, date_list, title_list)), columns = ['언론사','날짜', '제목'])
df

In [ ]:
# csv로 뽑기
df.to_csv('2017.csv')

In [100]:
driver.close()